In [1]:
import json
import json
import subprocess
from subprocess import Popen
import paperscraper

file_path = "/home/jakubmitura/projects/thyroid_knowledge_graph/literatura_thyroid.json"

with open(file_path, "r") as file:
    data = json.load(file)
data
# Now you can work with the loaded JSON data
# ...



[{'citation-number': ['\u202d27.\u202c'],
  'author': [{'family': 'Lauper', 'given': 'J.'},
   {'family': 'Krause', 'given': 'A.'},
   {'family': 'Vaughan', 'given': 'T.'},
   {'family': 'Monnat', 'given': 'R.'}],
  'title': ['Spectrum and risk of neoplasia in Werner\u202c \u202dsyndrome: a systematic review'],
  'volume': ['8'],
  'url': ['https://pubmed.ncbi.nlm.nih.gov/23573208/'],
  'type': 'article-journal',
  'container-title': ['PLoS One'],
  'date': ['2013'],
  'issue': ['4']},
 {'citation-number': ['\u202d34.\u202c'],
  'author': [{'family': 'Guilmette', 'given': 'J.'},
   {'family': 'Nosé', 'given': 'V.'}],
  'title': ['Hereditary and familial thyroid tumours'],
  'volume': ['\u202d72(1):70-81'],
  'doi': ['URL:\u202c\u202dhttps://pubmed.ncbi.nlm.nih.gov/29239041/\u202c'],
  'type': 'article-journal',
  'container-title': ['Histopathology'],
  'date': ['2018']},
 {'citation-number': ['\u202d106.\u202c'],
  'author': [{'family': 'Ahuja', 'given': 'A.'},
   {'family': 'Ying', '

In [2]:
from paperscraper.aggregators.doi_aggregator import DOIAggregator
from paperscraper.aggregators.pubmed_aggregator import PubMedAggregator
from paperscraper.scrapers.science_direct_scraper import ScienceDirect
from paperscraper.scrapers.acs_scraper import ACS
from paperscraper.scrapers.pmc_scraper import PMC
from paperscraper.scrapers.rsc_scraper import RSC
from habanero import Crossref

from selenium import webdriver
import pkg_resources
import time
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
import chromedriver_autoinstaller
import sys

class PaperScraper():
    """This class provides a direct interface to the inner functionality of paperscraper

    .. note::

       This is the only class needed to utilize all features of paperscraper.

    """

    def __init__(self, webdriver_path=None):
        """Creates a PaperScraper object

        Initializes a PaperScraper that can scrape text and meta-data from scientific journals. Individual journal
        scrapers and journal link aggregators are implemented in :mod:'scrapers' and :mod:'aggregators'.

        :param webdriver_path: The file path of a custom web driver to utilize, defaults to utilize the chromedriver
            that comes installed with the package.
        :type webdriver_path: str.

        """
        options = webdriver.ChromeOptions()
        options.add_argument('headless')

        webdriver_path = pkg_resources.resource_filename('paperscraper', 'webdrivers/chromedriver')

        if ('webdriver_path' is not None):
            self.webdriver_path = webdriver_path
            
        sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')



        # setup chrome options
        chrome_options = webdriver.ChromeOptions()
        chrome_options.add_argument('--headless') # ensure GUI is off
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--disable-dev-shm-usage')

        # set path to chromedriver as per your configuration
        chromedriver_autoinstaller.install()

        # set the target URL
        url = "put-url-here-to-scrape"

        # set up the webdriver
        driver = webdriver.Chrome(options=chrome_options)            
            

        # self.driver = webdriver.Chrome(webdriver_path, options=options)
        self.driver =driver

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.driver.quit()

    def __import_all_scrapers(self):
        return [ScienceDirect(self.driver), ACS(self.driver), PMC(self.driver), RSC(self.driver)]

    def get_scrapable_websites(self):
        """
        Retrieves all journal urls that are available to be scraped.
        :return:
        """
        return [website for scraper in self.__import_all_scrapers() for website in scraper.website]

    def is_scrapable(self, url):
        """
        Checks if a given url is scrapable by PaperScraper

        :param url: a url containing a full text that needs scrapping
        :return: the Scraper that can scrape url or None
        """

        for website_scraper in self.__import_all_scrapers():
            if (website_scraper.is_correct_url(url)):
                return website_scraper
        return None

    def extract_from_url(self, url):
        """
           Return a JSON file containing a the full text and meta data of the paper located at 'url'.
           Returns None if 'url' cannot be scraped.
           """
        for website_scraper in self.__import_all_scrapers():
            if (website_scraper.is_correct_url(url)):
                return website_scraper.extract(url)
        return None

    def extract_from_pmid(self, pmid):
        """
        Attempts to retrieve a paper given its PMID

        :param pmid:
        :return: An OrderedDict object containing the extracted paper
        """
        pm = PubMedAggregator(self.driver)
        all_sites = pm.extract(pmid)
        for url in [all_sites.get(key)['href'] for key in all_sites.keys()]:
            website_scraper = self.is_scrapable(url)
            if website_scraper is not None:
                return self.extract_from_url(url)

        return None

    def get_sites_from_pmid(self, pmid):
        pm = PubMedAggregator(self.driver)
        all_sites = pm.extract(pmid, follow_link=True)
        return [all_sites.get(key)['href'] for key in all_sites.keys()]

    def get_sites_from_doi(self, doi_num):
        dm = DOIAggregator()
        extracted_site = dm.extract(doi_num)
        return [extracted_site]


<>:43: SyntaxWarning: "is not" with a literal. Did you mean "!="?
<>:43: SyntaxWarning: "is not" with a literal. Did you mean "!="?
/tmp/ipykernel_372651/1222350082.py:43: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  if ('webdriver_path' is not None):


In [3]:
scraper = PaperScraper()
cr = Crossref()


In [6]:
import json
from Bio import Entrez

def get_pmid(doi):
    Entrez.email = "jakub.mitura14@gmail.com"  # Always tell NCBI who you are
    handle = Entrez.esearch(db="pubmed", term=doi)
    record = Entrez.read(handle)
    handle.close()
    if record["IdList"]:
        return record["IdList"][0]
    else:
        return None
    
def downl(ref):

    try:
        title=ref['title'][0]
        result = cr.works(query = title)
        doi=result['message']['items'][0]['DOI']
        print(f"doii {doi}")
        title= title.replace(" ", "_")
        path_out=f'/home/jakubmitura/projects/thyroid_knowledge_graph/autodownloaded'
        p = subprocess.Popen(f'python3 -m PyPaperBot --doi="{doi}" --dwn-dir={path_out}', shell=True)
        p.wait()
    except Exception as e:
        print(f"error {e}")
            
           


# extract_from_url
list(map(downl,data))

doii 10.1371/journal.pone.0059709
PyPaperBot is a Python tool for downloading scientific papers using Google Scholar, Crossref and SciHub.
If you like this project, you can give me a cup of coffee at --> https://www.paypal.com/paypalme/ferru97 <-- :)


Searching paper 1 of 1 with DOI 10.1371/journal.pone.0059709
Python 3

Using https://sci-hub.shop as Sci-Hub instance
Download 1 of 1 -> Spectrum and Risk of Neoplasia in Werner Syndrome: A Systematic Review

Work completed!
If you like this project, you can offer me a cup of coffee at --> https://www.paypal.com/paypalme/ferru97 <-- :)

doii 10.1111/his.13373
PyPaperBot is a Python tool for downloading scientific papers using Google Scholar, Crossref and SciHub.
If you like this project, you can give me a cup of coffee at --> https://www.paypal.com/paypalme/ferru97 <-- :)


Searching paper 1 of 1 with DOI 10.1111/his.13373
Python 3

Using https://sci-hub.shop as Sci-Hub instance
Download 1 of 1 -> Hereditary and familial thyroid tumours


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [ ]:
data[1]